# Extracting Data from the Morningstar Deliverable 

## Importing Libraries
For this script, we use Pandas and cElementTree which is a C based counterpart of ElementTree for XML scraping. We will use 'iterparse' as it is similar to the performance of SAX and basic Python I/O to minimize RAM usage.

In [1]:
# Importing Libraries
from alive_progress import alive_bar
import xml.etree.cElementTree as ET
import zipfile
import os

## Zip File Extraction
This function will open a data source path, unzip the datasets from specific folders, transfer them to a temporary folder called "Data_Unzipped" and output the coressponding path.

In [2]:
def extract_zipfiles (source_path, debug) :
    
    if debug : 
        unzip_path = '/Users/theoobadiahteguh/Desktop/Testing/DataUnzipped'
    else : 
        unzip_path = '/Users/theoobadiahteguh/Desktop/MorningStar/Output/DataUnzipped'
    
    # Check if the output folder has been created
    isExist = os.path.exists(unzip_path)

    # Creates a an output folder if needed
    if not isExist: 
        os.makedirs(unzip_path)
    
    # Iterates through the data source directory and unzips wanted files
    for folder in os.listdir(source_path):

        # Avoid certain hidden files that show up on Mac OS
        if folder == ".DS_Store" : 
            continue

        for item in os.listdir(f'{source_path}/{folder}') :
            if item.endswith('.xml.zip'):
                with zipfile.ZipFile(f'{source_path}/{folder}/{item}', 'r') as zip_ref:
                    zip_ref.extractall(unzip_path)
    
    return unzip_path

## Text File Creation
Filters all data into a .txt file for easier processing. This method will directly write to the disk and operates perfectly with low RAM.

In [3]:
def create_txtfile (iterable, Filename, debug) :
    
    if debug : 
        txtfile_path = '/Users/theoobadiahteguh/Desktop/Testing/TextFiles'
    else : 
        txtfile_path = '/Users/theoobadiahteguh/Desktop/MorningStar/Output/TextFiles'

    # Check if the output folder has been created
    isExist = os.path.exists(txtfile_path)

    # Creates a an output folder if needed
    if not isExist: 
        os.makedirs(txtfile_path)
    
    psx_handle = open(f"{txtfile_path}/PSX.txt", 'a')
    hdx_handle = open(f"{txtfile_path}/HDX.txt", 'a')
    
    del txtfile_path
    
    Section = None
    
    # The items needed for both data frames
    Portfolio_ORDINAL         = 0
    Fund_Name                 = None
    ShareClassId              = None # Use Investment Vehicle ID
    Date                      = None
    _CurrencyId               = None
    PortfolioSummary_ORDINAL  = 0
    #PreviousPortfolioDate    = None
    #NetExpenseRatio          = 0
    
    # All the items needed specifically for PortfolioSummaryX
    
    psx_cache = []
    
    NumberOfHoldingShort      = '0'
    NumberOfStockHoldingShort = '0'
    NumberOfBondHoldingShort  = '0'
    TotalMarketValueShort     = '0'
    NumberOfHoldingLong       = '0'
    NumberOfStockHoldingLong  = '0'
    NumberOfBondHoldingLong   = '0'
    TotalMarketValueLong      = '0'
    
    # All the items needed specifically for HoldingDetailX
    
    hdx_cache = []
    
    # HoldingOrdinal is assigned when entering a section
    _DetailHoldingTypeId             = None
    #_StorageId                      = None
    _ExternalId                      = None
    _Id                              = None
    #ExternalName                    = None
    Country_Id                       = None
    Country                          = None
    CUSIP                            = None
    SEDOL                            = None
    ISIN                             = None
    Ticker                           = None
    Currency                         = None
    Currency_Id                      = None
    SecurityName                     = None
    #LocalName                       = None
    Weighting                        = None
    NumberOfShare                    = '0'
    SharePercentage                  = None
    #NumberOfJointlyOwnedShare       = None
    MarketValue                      = '0'
    CostBasis                        = None
    ShareChange                      = '0'
    Sector                           = None
    MaturityDate                     = None
    #AccruedInterest                 = None
    Coupon                           = None
    CreditQuality                    = None
    Duration                         = None
    IndustryId                       = None
    #GlobalIndustryId                = None
    #GlobalSector                    = None
    #GICSIndustryId                  = None
    #LocalCurrencyCode               = None
    #LocalMarketValue                = None
    #ZAFAssetType                    = None
    PaymentType                      = None
    Rule144AEligible                 = None
    AltMinTaxEligible                = None
    #BloombergTicker                 = None
    #ISOExchangeID                   = None
    #ContractSize                    = None
    #SecondarySectorId               = None
    #CompanyId                       = None # Operation
    FirstBoughtDate                  = None
    #MexicanTipoValor                = None
    #MexicanSerie                    = None
    #MexicanEmisora                  = None
    #UnderlyingSecId                 = None # UnderlyingFundId in InvestmentVehicle/SubAccount
    #UnderlyingSecurityName          = None # NumberOfUnderlyingFund in PortfolioStatistics
    #PerformanceId                   = None # InvestmentVehicle/TrailingPerformance/MonthEndTrailingPerformance/PerformanceId
    LessThanOneYearBond              = None
    #ZAFBondIssuerClass               = None
    #IndianCreditQualityClassificatio = None
    #IndianIndustryClassification     = None
    #SurveyedWeighting                = None
    
    
    for event, element in iterable:
 
        tag = element.tag
        value = element.text
        
        if tag == "Portfolio" and event == "start" :
            _CurrencyId = element.attrib.get("_CurrencyId")
            Portfolio_ORDINAL += 1
        elif tag == "Portfolio" and event == "end" :
            Fund_Name           = None
            ShareClassId        = None
            Date                = None
            _CurrencyId         = None
        
        # Accessing a Portfolio Summary and saving data
        if tag == "PortfolioSummary" and event == "start" :
            PortfolioSummary_ORDINAL += 1
            
        elif tag == "PortfolioSummary" and event == "end" :
            tmp_list_PSX = [Filename,
                            Portfolio_ORDINAL,
                            Fund_Name,
                            ShareClassId,
                            Date,
                            _CurrencyId,
                            PortfolioSummary_ORDINAL,
                            NumberOfHoldingShort,      
                            NumberOfStockHoldingShort,
                            NumberOfBondHoldingShort, 
                            TotalMarketValueShort, 
                            NumberOfHoldingLong, 
                            NumberOfStockHoldingLong, 
                            NumberOfBondHoldingLong, 
                            TotalMarketValueLong]
        
            psx_cache.extend([';'.join(list(map(str, tmp_list_PSX)))])
            tmp_list_PSX = []
            
            if len(psx_cache) % 5000 == 0 :
                psx_handle.write('\n'.join([row for row in psx_cache]) + '\n')
                psx_handle.flush()
                psx_cache = []

            # Reset Values after Creating a Row
            NumberOfHoldingShort      = '0'
            NumberOfStockHoldingShort = '0'
            NumberOfBondHoldingShort  = '0'
            TotalMarketValueShort     = '0'
            NumberOfHoldingLong       = '0'
            NumberOfStockHoldingLong  = '0'
            NumberOfBondHoldingLong   = '0'
            TotalMarketValueLong      = '0'
        
        # Separating the different sections of the data
        if tag == "FundShareClass" and event == "start" :
            ShareClassId = element.attrib.get('_Id')
        
        if tag == "Name": Fund_Name = value

        if tag == "Date" : Date = value
        
        if tag == "HoldingAggregate" and event == "start" :
            if element.attrib.get("_SalePosition") == "L" :
                Section = "L"
            elif element.attrib.get("_SalePosition") == "S" :
                Section = "S"
        elif tag == "HoldingAggregate" and event == "end" :
            Section = None
        
        if tag == "Holding" and event == "start" :
            Holding_Ordinal = 0
        elif tag == "Holding" and event == "end" :
            del Holding_Ordinal
            
        if tag == "HoldingDetail" and event == "start" : 
            Section = "D"
            Holding_Ordinal += 1   
        elif tag == "HoldingDetail" and event == "end" :
            
            # Append row of data to list
            tmp_list_HDX = [Filename,
                            Portfolio_ORDINAL,
                            Fund_Name,
                            ShareClassId, 
                            Date,
                            _CurrencyId,
                            PortfolioSummary_ORDINAL,
                            Holding_Ordinal,
                            _DetailHoldingTypeId,
                            #_StorageId,
                            _ExternalId,
                            _Id,
                            #ExternalName,
                            Country_Id,
                            Country,
                            CUSIP,
                            SEDOL,
                            ISIN,
                            Ticker,
                            Currency,
                            Currency_Id,
                            SecurityName,
                            #LocalName,
                            Weighting,
                            NumberOfShare,
                            SharePercentage,
                            #NumberOfJointlyOwnedShare,
                            MarketValue,
                            CostBasis,
                            ShareChange,
                            Sector,
                            MaturityDate,
                            #AccruedInterest,
                            Coupon,
                            CreditQuality,
                            Duration,
                            IndustryId,
                            #GlobalIndustryId,
                            #GlobalSector,
                            #GICSIndustryId,
                            #LocalCurrencyCode,
                            #LocalMarketValue,
                            #ZAFAssetType,
                            PaymentType,
                            Rule144AEligible,
                            AltMinTaxEligible,
                            #BloombergTicker,
                            #ISOExchangeID,
                            #ContractSize,
                            #SecondarySectorId,
                            #CompanyId,
                            FirstBoughtDate,
                            #MexicanTipoValor,
                            #MexicanSerie,
                            #MexicanEmisora,
                            #UnderlyingSecId,
                            #UnderlyingSecurityName,
                            #PerformanceId,
                            LessThanOneYearBond]
            
            hdx_cache.extend([';'.join(list(map(str, tmp_list_HDX)))])
            tmp_list_HDX = []
            
            # Reset Values after Creating a Row
            _DetailHoldingTypeId             = None
            #_StorageId                      = None
            _ExternalId                      = None
            _Id                              = None
            #ExternalName                    = None
            Country_Id                       = None
            Country                          = None
            CUSIP                            = None
            SEDOL                            = None
            ISIN                             = None
            Ticker                           = None
            Currency                         = None
            Currency_Id                      = None
            SecurityName                     = None
            #LocalName                       = None
            Weighting                        = None
            NumberOfShare                    = '0'
            SharePercentage                  = None
            #NumberOfJointlyOwnedShare       = None
            MarketValue                      = '0'
            CostBasis                        = None
            ShareChange                      = '0'
            Sector                           = None
            MaturityDate                     = None
            #AccruedInterest                 = None
            Coupon                           = None
            CreditQuality                    = None
            Duration                         = None
            IndustryId                       = None
            #GlobalIndustryId                = None
            #GlobalSector                    = None
            #GICSIndustryId                  = None
            #LocalCurrencyCode               = None
            #LocalMarketValue                = None
            #ZAFAssetType                    = None
            PaymentType                      = None
            Rule144AEligible                 = None
            AltMinTaxEligible                = None
            #BloombergTicker                 = None
            #ISOExchangeID                   = None
            #ContractSize                    = None
            #SecondarySectorId               = None
            #CompanyId                       = None 
            FirstBoughtDate                  = None
            #MexicanTipoValor                = None
            #MexicanSerie                    = None
            #MexicanEmisora                  = None
            #UnderlyingSecId                 = None 
            #UnderlyingSecurityName          = None 
            #PerformanceId                   = None 
            LessThanOneYearBond              = None
            #ZAFBondIssuerClass               = None
            #IndianCreditQualityClassificatio = None
            #IndianIndustryClassification     = None
            #SurveyedWeighting                = None
            
            Section = None
            
            if len(hdx_cache) % 50000 == 0 :
                hdx_handle.write('\n'.join([row for row in hdx_cache]) + '\n')
                hdx_handle.flush()
                hdx_cache = []

        if Section == "L":
            if tag == "NumberOfHolding" : 
                if value == None :
                    NumberOfHoldingLong = '0'
                else :
                    NumberOfHoldingLong = value
            elif tag == "NumberOfStockHolding" :
                if value == None :
                    NumberOfStockHoldingLong = '0'
                else :
                    NumberOfStockHoldingLong = value
            elif tag == "NumberOFBondHolding" :
                if value == None :
                    NumberOfBondHoldingLong = '0'
                else :
                    NumberOfBondHoldingLong = value
            elif tag == "TotalMarketValue" :
                if value == None :
                    TotalMarketValueLong = '0'
                else :
                    TotalMarketValueLong = value
      
        elif Section == "S":
            if tag == "NumberOfHolding" : 
                if value == None :
                    NumberOfHoldingShort = '0'
                else :
                    NumberOfHoldingShort = value
            elif tag == "NumberOfStockHolding" :
                if value == None : 
                    NumberOfStockHoldingShort = '0'
                else :
                    NumberOfStockHoldingShort = value
            elif tag == "NumberOFBondHolding" :
                if value == None :
                    NumberOfBondHoldingShort = '0'
                else :
                    NumberOfBondHoldingShort = value
            elif tag == "TotalMarketValue" :
                if value == None :
                    TotalMarketValueShort = '0'
                else :
                    TotalMarketValueShort = value
                  
        elif Section == "D":
            if tag == "DetailHoldingTypeId" : _DetailHoldingTypeId = value
            elif tag == "HoldingDetail" :
                _ExternalId = element.attrib.get("_ExternalId")
                _Id = element.attrib.get("_Id")
            elif tag == "Country" :
                Country_Id = element.attrib.get("_Id")
                Country = value
            elif tag == "CUSIP"  : CUSIP  = value
            elif tag == "SEDOL"  : SEDOL  = value
            elif tag == "ISIN"   : ISIN   = value
            elif tag == "Ticker" : Ticker = value
            elif tag == "Currency" :
                Currency = value
                Currency_Id = element.attrib.get("_Id")
            elif tag == "SecurityName"        : SecurityName        = value
            elif tag == "Weighting"           : Weighting           = value
            elif tag == "NumberOfShare"       : 
                if value == None : NumberOfShare = '0'
                else : NumberOfShare = value
            elif tag == "SharePercentage"     : SharePercentage     = value
            elif tag == "MarketValue"         :
                if value == None : MarketValue = '0'
                else : MarketValue = value
            elif tag == "CostBasis"           : CostBasis           = value
            elif tag == "ShareChange"         :
                if value == None : ShareChange = '0'
                else : ShareChange = value
            elif tag == "Sector"              : Sector              = value
            elif tag == "MaturityDate"        : MaturityDate        = value
            elif tag == "Coupon"              : Coupon              = value
            elif tag == "CreditQuality"       : CreditQuality       = value
            elif tag == "Duration"            : Duration            = value
            elif tag == "IndustryId"          : IndustryId          = value
            elif tag == "PaymentType"         : PaymentType         = value
            elif tag == "Rule144AEligible"    : Rule144AEligible    = value
            elif tag == "AltMinTaxEligible"   : AltMinTaxEligible   = value
            elif tag == "FirstBoughtDate"     : FirstBoughtDate     = value
            elif tag == "LessThanOneYearBond" : LessThanOneYearBond = value
            
        # It's important to clear the elements when working with big datasets
        if event == "end": element.clear()
        
    if len(psx_cache) != 0 :
        psx_handle.write('\n'.join([row for row in psx_cache]) + '\n')
        psx_handle.flush()
        psx_cache = []

    if len(hdx_cache) != 0 :
        hdx_handle.write('\n'.join([row for row in hdx_cache]) + '\n')
        hdx_handle.flush()
        hdx_cache = []
        
    psx_handle.close()
    hdx_handle.close()

## Main Body
Simply iterates through a given directory for all XML files. The code then processes the data for each file. There is also a progress bar for the amount of files read and processed.

In [4]:
def main() :
    
    debug = 1 # Set to 1 for debugging
    
    if debug : 
        source_path = '/Users/theoobadiahteguh/Desktop/Projects/MorningStar/project_data/Tests'
    else :
        source_path = '/Users/theoobadiahteguh/Desktop/MorningStar/project_data/Data'
    
    data_path = extract_zipfiles(source_path, debug)
    
    with alive_bar(len(os.listdir(data_path)) - 1 , force_tty = True) as bar:
        print(f"Loading datasets...")
        
        for filename in os.listdir(data_path):
            
            # Avoid certain hidden files that show up on Mac OS
            if filename == "__MACOSX" or filename == ".DS_Store" : 
                continue
            
            print(f"The file named '{filename}' is being processed.")
            fullname = os.path.join(data_path, filename)
            
            # Events tell whether it's an opening or closing tag
            context = ET.iterparse(fullname, events = ('start', 'end'))
            context = iter(context)
            
            create_txtfile(context, filename, debug)
            
            print(f"The file named '{filename}' has successfully been processed.")
            bar()
            

## Execution

In [5]:
main()

on 0: Loading datasets...                                                       
on 0: The file named 'example_5.xml' is being processed.                        
on 0: The file named 'example_5.xml' has successfully been processed.           ▆ 0/5 [0%] in 1:13 (~0s, 0.0/s)  ▃▅▇ 0/5 [0%] in 1:23 (~0s, 0.0/s)  ▄▂▂ 0/5 [0%] in 2:17 (~0s, 0.0/s) 
on 1: The file named 'example_4.xml' is being processed.                        
on 1: The file named 'example_4.xml' has successfully been processed.           
on 2: The file named 'example_3.xml' is being processed.                        
on 2: The file named 'example_3.xml' has successfully been processed.           
on 3: The file named 'example_2.xml' is being processed.                        
on 3: The file named 'example_2.xml' has successfully been processed.           
on 4: The file named 'example_1.xml' is being processed.                        
on 4: The file named 'example_1.xml' has successfully been processed.           
|██████